## Entrenamiento inicial del modelo y búsqueda de hiperparámetros

Para realizar este notebook se utilizó como base el ejemplo presentado en el curso para el dataset "Heart Disease", incluyendo algunas modificaciones para adaptarlo al nuevo dataset "Rain in Australia".

El objetivo de este notebook es realizar experimentos sobre el modelo para luego llevarlo ambiente de producción en caso de que supere ciertas pruebas y además sea aprobado por quien corresponda en la organización.

Para que el entrenamiento funcione localmente en un tiempo razonable se definió la variable *n_samples_train* que permite indicar qué tamaño tiene el subconjunto a entrenar, de forma de no utilizar todos los datos. Esto debe definirse sólo a modo de prueba para aquellos casos en que no se cuente con los recursos computacionales necesarios para procesar el set completo. En caso de que *n_samples_train* sea 0 se ignorará este parámetro.

In [49]:
import awswrangler as wr

import mlflow
from mlflow import MlflowClient, MlflowException
from mlflow.models import infer_signature
import optuna

from sklearn.tree import DecisionTreeClassifier

from mlflow_aux import get_or_create_experiment
from optuna_aux import champion_callback, objective
from plots import plot_correlation_with_target, plot_information_gain_with_target, plot_feature_correlation

from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import random
import datetime

%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

# Cantidad de samples a utilizar para el entrenamiento
n_samples_train = 1000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [50]:
# Se setea URL de seguimiento para MLflow
mlflow_server = "http://localhost:5000"
mlflow.set_tracking_uri(mlflow_server)

Se cargan los datos de entrenamiento y validación desde el bucket S3. A este punto se asume que ya corrió sin errores el DAG *etl_process_rain_australia*, haciendo las transformaciones necesarias de los datos.

In [51]:
# Se cargan los conjuntos de entrenamiento y validación.

X_train =  wr.s3.read_csv("s3://data/final/train/weather_X_train.csv")
y_train =  wr.s3.read_csv("s3://data/final/train/weather_y_train.csv")

X_test =  wr.s3.read_csv("s3://data/final/test/weather_X_test.csv")
y_test =  wr.s3.read_csv("s3://data/final/test/weather_y_test.csv")

In [52]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106644 entries, 0 to 106643
Data columns (total 28 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   MinTemp          106644 non-null  float64
 1   MaxTemp          106644 non-null  float64
 2   Rainfall         106644 non-null  float64
 3   Evaporation      106644 non-null  float64
 4   Sunshine         106644 non-null  float64
 5   WindGustSpeed    106644 non-null  float64
 6   WindSpeed9am     106644 non-null  float64
 7   WindSpeed3pm     106644 non-null  float64
 8   Humidity9am      106644 non-null  float64
 9   Humidity3pm      106644 non-null  float64
 10  Pressure9am      106644 non-null  float64
 11  Pressure3pm      106644 non-null  float64
 12  Cloud9am         106644 non-null  float64
 13  Cloud3pm         106644 non-null  float64
 14  Temp9am          106644 non-null  float64
 15  Temp3pm          106644 non-null  float64
 16  RainToday        106644 non-null  floa

Previo a la elaboración del DAG *etl_process_rain_australia* ya se realizó un análisis y limpieza del dataset en el que, entre otras acciones, se determinó las correlación entre las variables de entrada y de éstas con la variable objetivo. De todas formas, es útil registrar esta información para observar en el tiempo los posibles cambios que se puedan producir.

In [53]:
# Se obtienen los gráficos correspondientes a las correlaciones
correlation_plot = plot_correlation_with_target(X_train, y_train, target_col='RainTomorrow')
information_gain_plot = plot_information_gain_with_target(X_train, y_train, target_col='RainTomorrow')
features_correlation_plot = plot_feature_correlation(X_train)

Las diferentes ejecuciones de entrenamiento quedarán registradas en el experimento "Rain in Australia". El tuneo de hiperparámetros se mostrará como un *run* dentro de este experimentos que a su vez contendrá *runs* hijos correspondientes a cada prueba (*trial*) realizada.

In [54]:
experiment_id = get_or_create_experiment("Rain in Australia")
print(experiment_id)

run_name_parent = "best_hyperparam_"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

1


Tal como se comentó al comienzo, en caso de estar definida la variable *n_samples_train* se tomará un subconjunto de datos. La idea de utilizar esto es sólo para pruebas en equipos con bajos recursos computacionales.

In [55]:
if n_samples_train > 0:

    indices = random.sample(range(len(X_train)), n_samples_train)

    X_train = X_train.iloc[indices].reset_index(drop=True)
    y_train = y_train.iloc[indices].reset_index(drop=True)

Se realiza la optimización usan Optuna. Además, como se indicó anteriormente, se reutilizaron las librerías del ejemplo proporcionado en el curso dado que simplifican mucho la tarea y proporcionan una base sólida para futuras mejoras.

In [56]:
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    # Inicializamos el estudio de Optuna
    study = optuna.create_study(direction="maximize")

    # Ejecutamos los trials de optimización de hiperparametros. Cada uno de estos trials se ejecuta con un run separado, pero 
    # está anidado al run padre.
    # Notar la adición del `champion_callback` para controlar qué mensajes mostramos
    # Para entender mejor esto ver la documentación de objective y champion_callback en optuna_aux
    study.optimize(lambda trial: objective(trial, X_train, y_train, experiment_id), n_trials=250, callbacks=[champion_callback])

    # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_train_f1", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Rain in Australia",
            "optimizer_engine": "optuna",
            "model_family": "sklearn",
            "feature_set_version": 1,
        }
    )

    # Una vez que terminamos la búsqueda, nos quedamos con el mejor modelo y lo entrenamos
    if study.best_params["classifier"] == "SVC_linear":
        model = SVC(C=study.best_params["svc_c"], kernel='linear', gamma='scale')
    elif study.best_params["classifier"] == "SVC_poly":
        model = SVC(C=study.best_params["svc_c"], kernel='poly', 
                    gamma='scale', degree=study.best_params["svc_poly_degree"])
    elif study.best_params["classifier"] == "SVC_rbf":
        model = SVC(C=study.best_params["svc_c"], kernel='rbf', gamma='scale')
    elif study.best_params["classifier"] == "DecisionTreeClassifier":
        model = DecisionTreeClassifier(max_depth=study.best_params["tree_max_depth"])
    else:
        model = RandomForestClassifier(max_depth=study.best_params["rf_max_depth"], 
                                       n_estimators=study.best_params["rf_n_estimators"])

    model = model.fit(X_train, y_train.to_numpy().ravel())

    # Y testeamos el modelo y logueamos el resultado
    y_pred = model.predict(X_test)
    f1_score = f1_score(y_test.to_numpy().ravel(), y_pred)
    mlflow.log_metric("test_f1", f1_score)

    # Logueamos los artefactos de las gráficas de correlación y de information_gain
    mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")
    mlflow.log_figure(figure=information_gain_plot, artifact_file="information_gain_plot.png")
    mlflow.log_figure(features_correlation_plot, artifact_file="features_correlation_plot.png")

    # Guardamos el artefacto del modelo
    artifact_path = "model"

    signature = infer_signature(X_train, model.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="rain_australia_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)

[I 2024-08-24 18:33:39,444] A new study created in memory with name: no-name-eaffba1f-7335-4b56-909b-b00e39598f93
2024/08/24 18:33:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 0 at: http://localhost:5000/#/experiments/1/runs/99fc7b7cafbd422b8661c17f1eacaedf.
2024/08/24 18:33:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:33:52,898] Trial 0 finished with value: 0.4602099530519091 and parameters: {'classifier': 'SVC_rbf', 'svc_c': 53.17915624621662}. Best is trial 0 with value: 0.4602099530519091.


Initial trial 0 achieved value: 0.4602099530519091


2024/08/24 18:33:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 1 at: http://localhost:5000/#/experiments/1/runs/2e29a9b006a1410aab9bedad4cd9d93c.
2024/08/24 18:33:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:33:53,455] Trial 1 finished with value: 0.5054419635157734 and parameters: {'classifier': 'SVC_linear', 'svc_c': 0.2971184033131538}. Best is trial 1 with value: 0.5054419635157734.


Trial 1 achieved value: 0.5054419635157734 with  8.9490% improvement


2024/08/24 18:33:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 2 at: http://localhost:5000/#/experiments/1/runs/147c81629561479b8110270861be11f7.
2024/08/24 18:33:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:33:54,134] Trial 2 finished with value: 0.0 and parameters: {'classifier': 'SVC_rbf', 'svc_c': 0.05786627489993669}. Best is trial 1 with value: 0.5054419635157734.
2024/08/24 18:33:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 3 at: http://localhost:5000/#/experiments/1/runs/bb6b233d50174462a86c919b828e6550.
2024/08/24 18:33:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:33:55,014] Trial 3 finished with value: 0.3927337382875125 and parameters: {'classifier': 'DecisionTreeClassifier', 'tree_max_depth': 5}. Best is trial 1 with value: 0.5054419635157734.
2024/08/24 18:33:56 INF

...

2024/08/24 18:34:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 17 at: http://localhost:5000/#/experiments/1/runs/8a1c87910aff4dc79b5dbd31c4c455e7.
2024/08/24 18:34:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:34:48,171] Trial 17 finished with value: 0.44375732269388984 and parameters: {'classifier': 'SVC_linear', 'svc_c': 0.01388673778958273}. Best is trial 16 with value: 0.5104138551506973.
2024/08/24 18:34:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 18 at: http://localhost:5000/#/experiments/1/runs/7496e983cdbd42a89761724a92cecddf.
2024/08/24 18:34:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:34:49,159] Trial 18 finished with value: 0.5104138551506973 and parameters: {'classifier': 'SVC_linear', 'svc_c': 1.0826260515354362}. Best is trial 16 with value: 0.5104138551506973.
20

Trial 24 achieved value: 0.5120688059161003 with  0.3232% improvement


2024/08/24 18:34:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 25 at: http://localhost:5000/#/experiments/1/runs/edc8f34e00bf49f9a642eff1c87aee1b.
2024/08/24 18:34:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:34:54,890] Trial 25 finished with value: 0.4138188608776844 and parameters: {'classifier': 'DecisionTreeClassifier', 'tree_max_depth': 32}. Best is trial 24 with value: 0.5120688059161003.
2024/08/24 18:34:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 26 at: http://localhost:5000/#/experiments/1/runs/77571a9f7bc8457e9d5d92712b761c73.
2024/08/24 18:34:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:34:55,459] Trial 26 finished with value: 0.5120688059161003 and parameters: {'classifier': 'SVC_linear', 'svc_c': 0.520563236825388}. Best is trial 24 with value: 0.5120688059161003.


Trial 104 achieved value: 0.5133675072148016 with  0.2530% improvement


2024/08/24 18:35:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 105 at: http://localhost:5000/#/experiments/1/runs/a422dff6e2594c54bc632d53867cb702.
2024/08/24 18:35:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:35:50,186] Trial 105 finished with value: 0.5055713034185977 and parameters: {'classifier': 'SVC_linear', 'svc_c': 0.6354613677409721}. Best is trial 104 with value: 0.5133675072148016.
2024/08/24 18:35:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 106 at: http://localhost:5000/#/experiments/1/runs/ce0b6ee5cc364beaa6885562bb0ea1e4.
2024/08/24 18:35:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-24 18:35:50,923] Trial 106 finished with value: 0.5067739648477747 and parameters: {'classifier': 'SVC_linear', 'svc_c': 0.35589662763430063}. Best is trial 104 with value: 0.513367507214801

Una vez entrenado el modelo, en este punto se deberían realizar diferentes pruebas para validar que se encuentre en condiciones de ser puesto en producción. Una vez recibida la aprobación, se deberían ejecutar los siguientes pasos para promover el modelo.

In [57]:
client = MlflowClient()

In [58]:
# Se define el nombre del modelo y su descripción
name = "rain_australia_model_prod"
desc = "This classifier predicts whether it will rain tomorrow"

En caso que no sea la primera vez que se corre este notebook y ya existe un modelo en producción, se elimina el modelo anterior antes de crear el nuevo.

In [59]:
try:
    # Intenta obtener el modelo registrado por nombre
    registered_model = client.get_registered_model(name=name)
    
    # Si se encuentra, procede a eliminarlo
    client.delete_registered_model(name=name)
    print(f"El modelo '{name}' ha sido eliminado.")
    
except MlflowException as e:
    # Captura la excepción si el modelo no existe
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        print(f"El modelo '{name}' no existe. No se realizó ninguna acción.")
    else:
        # Si es otra excepción, la relanza
        raise

El modelo 'rain_australia_model_prod' ha sido eliminado.


Se crea el modelo productivo en MLflow. Se le asigna el alias "champion" que indica que es el mejor hasta el momento, pero que eventualmente podría ser sustituido por otro modelo "challenger" entrenado en el dag *train_model_rain_australia* agendado en Airflow. En este modelo se almacenan como tags los metadatos del mismo así como también el resultado de la métrica *f1-score*.

In [60]:
# Creamos el modelo productivo
client.create_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = model.get_params()
tags["model"] = type(model).__name__
tags["f1-score"] = f1_score

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "champion", result.version)

2024/08/24 18:37:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: rain_australia_model_prod, version 1
